# 2. Use Softmax--->2顆神經元
## (更改輸出層)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
data = pd.read_csv('diabetes.csv')  

# 分割数据为特征 (X) 和标签 (y)
X = data.iloc[:, 1:-1]  
y = data['Diabetic'].values.reshape(-1, 1)
# ohe = OneHotEncoder()
# ohe.fit_transform(y)[:3]
encoded = to_categorical(y).astype('int')
encoded

### (to_categorical(y) / np.argmax( ))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers


data = pd.read_csv('diabetes.csv')  

X = data.iloc[:, 1:-1]  
y = data['Diabetic'].values.reshape(-1, 1)
y = to_categorical(y).astype('int') # 因為輸出的時候分成 [0, 1] 對應機率

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = keras.Sequential()
model.add(layers.Input(shape=(X_train.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))  # 二元分类问题，使用 softmax 激活函数
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, batch_size=16, verbose = 2, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f'测试集上的损失：{loss}')
print(f'测试集上的准确率：{accuracy}')


In [ ]:
print(history.history.keys())
history.history['accuracy'][:3]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 3))
plt.plot(history.history['accuracy'], color = 'red')
plt.plot(history.history['val_accuracy'], color = 'blue')
plt.grid()
plt.title('accuracy vs val_accuracy')
plt.show()

import matplotlib.pyplot as plt
plt.figure(figsize=(12, 3))
plt.plot(history.history['loss'], color = 'red')
plt.plot(history.history['val_loss'], color = 'blue')
plt.grid()
plt.title('accuracy vs val_accuracy')
plt.show()

In [ ]:
# 预测新数据
import warnings
warnings.filterwarnings("ignore")

new_data = np.array([[10, 70, 30, 40, 20, 35, 0.5, 30]])  # 替换为你的新数据
new_data = scaler.transform(new_data)
prediction = model.predict(new_data)
print(np.argmax(prediction, axis=1))
print(f'预测结果：{np.argmax(prediction, axis=1)[0]}')

### 請輸出類似以下的結果 :
### PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age, Diabetic(0, 1), Predict(0, 1),Accuracy(正確, 錯誤)


In [ ]:
import numpy as np
data = pd.read_csv('diabetes.csv')
all_data = data.iloc[:, 1:-1].values
all_data = scaler.transform(all_data)
all_prediction = model.predict(all_data) # 因為最後一顆神經元是2個 softmax輸出
data['Predict'] = np.argmax(all_prediction, axis = 1)
data[:3]
# data['Predict'] = np.where( (all_prediction.ravel() >= 0.5) == True, 1, 0)
# data[:3]  #將所有預測轉換成一階張量後 判斷生病機率是否大(等)於 0.5 若就是有病 否則就是正常
#           #np.where 針對陣列批次轉換成為另外顯示資料，類似單一值 if else


In [ ]:
data['Accuracy'] = (data['Diabetic']==data['Predict'] )
data['Accuracy'] = data['Accuracy'].apply(lambda x : '正確' if x == True else '錯誤')
data[:3]

In [ ]:
data.to_excel('diabetes-softmax.xlsx', index = None)